# Day 12 - Cellular automata

It's a 1D [cellular automata](https://en.wikipedia.org/wiki/Cellular_automaton); like Conway's Game of Life but on a line rather than on a grid.

We could use numpy again, but since this is an infinite grid, I thought I'd give a linked list a try. The linked list is centered on coordinate 0, then fans out in either direction. Running a step then replaces old state with new, passing old state out to either side to update the next nodes.

To ensure performance and unlimited depth, I do want to avoid recursion here.

In [1]:
import re

from dataclasses import dataclass, field
from collections import deque
from itertools import chain, islice, product, repeat, tee
from operator import attrgetter
from typing import Callable, Dict, Iterator, Iterable, Mapping, List, Optional, Tuple

Dir = str
Plant = bool
_NextFunc = Callable[['Pot'], 'Pot']
_next_map: Mapping[str, _NextFunc] = {d: attrgetter(d) for d in ('left', 'right')}

@dataclass
class Pot:
    __slots__ = ('id', 'plant', 'left', 'right')

    id: int
    plant: Plant

    def __post_init__(self) -> None:
        self.left: Optional[Pot] = None
        self.right: Optional[Pot] = None

    def iter_dir(self, dir: Dir) -> Iterator['Pot']:
        """Iterate over the chain in the given direction
        
        The current pot is *not* included.
        
        """
        next_ = _next_map[dir]
        p = next_(self)
        while p:
            yield p
            p = next_(p)
    
    def with_potentials(self, dir: Dir) -> Iterator['Pot']:
        """Same as iter_dir but with PotentialPot added at the end"""
        p = self
        for p in self.iter_dir(dir):
            yield p
        potpot = PotentialPot(p.id, False, p, dir)
        yield potpot
        yield from potpot.with_potentials(dir)
        
    def __str__(self) -> str:
        return '#' if self.plant else '.'

    def append(self, dir: Dir, pot: 'Pot'):
        assert getattr(self, dir) is None
        setattr(self, dir, pot)
        setattr(pot, _reverse[dir], self)
        return pot
            
    @classmethod
    def from_initial(cls, initial: str) -> Tuple['Pot', int]:
        """Create pots from initail .# string
        
        Returns head of new chain plus length
        """
        # empty node to attach root and rest of chain to
        start = pot = cls(-1, False)
        for c in initial:
            pot = pot.append('right', cls(pot.id + 1, c == '#'))
        root, root.left = start.right, None
        return root, pot.id + 1

@dataclass
class PotentialPot(Pot):
    """Pot that is not yet linked into the chain
    
    When .plant is set to True, will become a permanent linked pot.
    
    """
    parent: Pot
    dir: Dir
    
    def __post_init__(self):
        super().__post_init__()
        self._linked: Optional[Pot] = None
        # make it easy to go back
        setattr(self, _reverse[self.dir], self.parent)

    @property
    def plant(self) -> Plant:
        if self._linked:
            return self._linked.plant
        return False
    
    @plant.setter
    def plant(self, plant: Plant) -> None:
        if not hasattr(self, '_linked'):
            # initialising, just ignore this one
            return
        if self._linked:
            self._linked.plant = plant
        elif plant:
            # link in new node; determine new id first
            self.id += 1 if self.dir == 'right' else -1
            self._linked = type(self.parent)(self.id, plant)
            self.parent.append(self.dir, self._linked)
    
    def with_potentials(self, dir: Dir) -> Iterator['Pot']:
        if self._linked:
            yield from self._linked.with_potentials(dir)

_rule_parse = re.compile('[.#]').findall
_State = List[Plant]
_reverse: Mapping[Dir, Dir] = {'left': 'right', 'right': 'left'}
_for_d: Iterable[Tuple[Dir, Callable[[str, _State], _State]]] = (
    # direction and state rotation function
    # s[:4] and s[-4:] make it possible to use 
    # the rotation functions to grow the central state too
    ('left', (lambda s, p: [p, *s[:4]])),
    ('right', (lambda s, p: [*s[-4:], p])),
)   

_RuleKey = Tuple[Plant, Plant, Plant, Plant, Plant]
_blank_rules: Mapping[_RuleKey, Plant] = dict.fromkeys(product([False, True], repeat=5), False)
        
@dataclass
class Rules:
    mapping: Dict[_RuleKey, Plant]

    @classmethod
    def from_lines(cls, lines: Iterable[str]) -> 'Rules':
        parsed = ((c == '#' for c in _rule_parse(line)) for line in lines if line.strip())
        return cls({**_blank_rules, **{tuple(k): r for *k, r in parsed}})
    
    def __str__(self) -> str:
        lines = []
        for k, p in self.mapping.items():
            *ks, ps = map('.#'.__getitem__, [*k, p])
            lines.append(f"{''.join(ks)} => {ps}")
        return '\n'.join(lines)
    
    def apply(self, root: Pot) -> Tuple[int, int]:
        """Apply rules to pots to produce an updated sequence
        
        Returns new length and summed pot ids for pots with plants
        """
        next_plant_state = self.mapping.__getitem__
        extremes = {'left': root.id, 'right': root.id}
        summed = 0  # root pot never counts        
        
        # build state map for central location, and lookahead iterators
        state: List[Plant] = [root.plant]
        lookaheads = {}
        for dir, rot in _for_d:
            lookahead = chain(
                (p.plant for p in root.iter_dir(dir)),
                repeat(False)
            )
            for plant in islice(lookahead, None, 2):
                state = rot(state, plant)
            lookaheads[dir] = lookahead
        central_state = state
            
        # update the root value
        root.plant = next_plant_state(tuple(state))

        # then go either direction
        for dir, rot in _for_d:
            lookahead = lookaheads[dir]
            
            state = central_state
            pot = root
            for pot, nextplant in zip(pot.with_potentials(dir), lookahead):
                extremes[dir] = pot.id

                # update state with plant from 2 pots down
                state = rot(state, nextplant)

                # update this state
                pot.plant = next_plant_state(tuple(state))
                summed += pot.id if pot.plant else 0                    
        
            # trim back down to last True value
            rev = _reverse[dir]
            assert isinstance(pot, PotentialPot)
            pot = pot.parent
            backward = pot.iter_dir(rev)
            while not pot.plant and pot is not root:
                todelete, pot = pot, next(backward)
                setattr(todelete, rev, None)
                setattr(pot, dir, None)
        
        return extremes['left'], extremes['right'] - extremes['left'] + 1, summed

@dataclass
class Plants:
    root: Pot
    length: int
    rules: Rules
                         
    def __post_init__(self):
        self.first: int = 0
    
    @classmethod
    def from_lines(cls, lines: Iterable[str]) -> 'Plants':
        it = iter(lines)
        initial = next(it).partition('initial state:')[-1].strip()
        return cls(*Pot.from_initial(initial), Rules.from_lines(it))

    def __len__(self):
        return self.length
    
    def __str__(self) -> str:
        root = self.root
        chars = [str(root), *map(str, root.iter_dir('left'))][::-1]
        chars += map(str, root.iter_dir('right'))
        return ''.join(chars)

    def step(self) -> int:
        self.first, self.length, summed = self.rules.apply(self.root)
        return summed


In [2]:
testplants = Plants.from_lines('''\
initial state: #..#.#..##......###...###

...## => #
..#.. => #
.#... => #
.#.#. => #
.#.## => #
.##.. => #
.#### => #
#.#.# => #
#.### => #
##.#. => #
##.## => #
###.. => #
###.# => #
####. => #'''.splitlines())
states = [(
    testplants.first,
    str(testplants),
    sum(p.plant * p.id for p in testplants.root.iter_dir('right')))
]
for _ in range(20):
    summed = testplants.step()
    states.append((testplants.first, str(testplants), summed))
max_length = max(len(s[1]) for s in states)
min_first = min(s[0] for s in states)
for i, (first, state, summed) in enumerate(states):
    leftpad = '.' * (first - min_first)
    rightpad = '.' * (max_length - len(state) - len(leftpad))
    print(f"{i:>2d}: {leftpad}{state}{rightpad} - {summed}")

 0: ..#..#.#..##......###...###.......... - 145
 1: ..#...#....#.....#..#..#..#.......... - 91
 2: ..##..##...##....#..#..#..##......... - 132
 3: .#.#...#..#.#....#..#..#...#......... - 102
 4: .#.#..#...#.#...#..#..##..##......... - 154
 5: ...#...##...#.#..#..#...#...#........ - 115
 6: ...##.#.#....#...#..##..##..##....... - 174
 7: ..#..###.#...##..#...#...#...#....... - 126
 8: ..#....##.#.#.#..##..##..##..##...... - 213
 9: ..##..#..#####....#...#...#...#...... - 138
10: .#.#..#...#.##....##..##..##..##..... - 213
11: .#...##...#.#...#.#...#...#...#...... - 136
12: ..##.#.#....#.#...#.#..##..##..##.... - 218
13: .#..###.#....#.#...#....#...#...#.... - 133
14: .#....##.#....#.#..##...##..##..##... - 235
15: .##..#..#.#....#....#..#.#...#...#... - 149
16: #.#..#...#.#...##...#...#.#..##..##.. - 226
17: #...##...#.#.#.#...##...#....#...#... - 170
18: .##.#.#....#####.#.#.#...##...##..##. - 280
19: #..###.#..#.#.#######.#.#.#..#.#...#. - 287
20: #....##....#####...#######....#.#..##

In [3]:
import aocd

data = aocd.get_data(day=12, year=2018)

In [4]:
plants = Plants.from_lines(data.splitlines())
for _ in range(20):
    summed = plants.step()
print('Part 1:', summed)

Part 1: 3725
